In [ ]:
# [Cell 1] - Set up paths
import os
from google.colab import drive
drive.mount('/content/drive')

# !unzip "/content/drive/MyDrive/cv-22928-2025-a-project.zip" -d "/content/"

BASE_PATH = "/content/drive/MyDrive"  # This will use the current directory
project_folder = "cv-22928-2025-a-project/train"
src = os.path.join(BASE_PATH, project_folder)

if os.path.exists(src):
    print(f"Successfully found project directory at: {src}")
else:
    print(f"ERROR: Could not find project directory at: {src}")
    print(f"Current working directory: {os.getcwd()}")
    print(f"Please check if the path '{src}' is correct")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Successfully found project directory at: /content/drive/MyDrive/cv-22928-2025-a-project/train


In [ ]:
!pip install numpy
!pip install opencv-python voxelmorph
!pip install matplotlib
!pip install tqdm
!pip install torch torchvision

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 9.9 MB/s eta 0:00:00
  Created wheel for pystrum: filename=pystrum-0.4-py3-none-any.whl size=19533 sha256=2714afb5ba72376ca845491ffaaa45a31a397a2ce5b8296eef6ed5551f192af3
  Stored in directory: /root/.cache/pip/wheels/7d/55/bd/7ac4dce19665c612e18993f517312c7ebabeaafed479bc4aac
Successfully built pystrum
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:

import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import Dataset
from torchvision import models
import os

# Define transformations (optional)
transform = transforms.Compose([
    transforms.Resize((128, 128)), # Resize all images to 128x128
    transforms.ToTensor(), # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # Normalize using ImageNet stats
    transforms.functional.rgb_to_grayscale
])

# Load the dataset using ImageFolder
dataset = datasets.ImageFolder(root=src, transform=transform)

# Create a DataLoader to iterate through the dataset in batches
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [10]:
import pytorch_lightning as pl
# from torchir.networks import DIRNet
from torchir.metrics import NCC
from torchir.transformers import BsplineTransformer

ModuleNotFoundError: No module named 'pytorch_lightning'

In [ ]:
for images, labels in dataloader:
    print(images.shape) # Shape of the batch (e.g., [32, 3, 128, 128])
    print(labels.shape) # Shape of the labels (e.g., [32])
    print(labels)
    break # Remove this break to iterate through the entire dataset

torch.Size([32, 1, 128, 128])
torch.Size([32])
tensor([ 5,  9,  2,  5,  0,  8,  2,  4,  0,  9,  2,  9,  3,  0,  3,  8,  2,  7,
         9,  4,  1,  3,  5,  5,  6,  5,  6, 11,  5,  2,  6,  0])


In [ ]:

Define the VoxelMorph model (for non-rigid registration)
model = VxmDense(inshape=image_shape, nb_unet_features=[32, 64, 128, 256])

Optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4)

Loss function (Mean Squared Error for simplicity)
def registration_loss(fixed, warped, warping_field):
    return mseloss(warped, fixed)  # Using MSE loss as an example

Training loop
num_epochs = 10  # Example number of epochs
for epoch in range(num_epochs):
    epoch_loss = 0.0
    for fixed, moving in dataloader:
        # Ensure fixed and moving are in correct device format (e.g., CUDA if available)
        fixed = fixed.cuda() if torch.cuda.is_available() else fixed
        moving = moving.cuda() if torch.cuda.is_available() else moving

        optimizer.zero_grad()

        # Forward pass: Register the moving image to the fixed image